In [1]:
from sea_breeze import load_model_data, sea_breeze_funcs, sea_breeze_filters
from dask.distributed import Client
from dask.distributed import progress
import pandas as pd
import os
import metpy.calc as mpcalc
import argparse

In [2]:
client = GadiClient()
client

NameError: name 'GadiClient' is not defined

In [ ]:
t1="2016-01-01 00:00"
t2="2016-01-31 23:00"
lat_slice=slice(-45.7,-6.9)
lon_slice=slice(108,158.5)

In [ ]:
chunks = {"time":-1,"lat":{},"lon":{}}
orog, lsm = load_model_data.load_aus2200_static(""
    "mjo-elnino",
    lon_slice,
    lat_slice)
era5_uas = load_model_data.load_era5_variable(
        ["10u"],
        t1,
        t2,
        lon_slice,
        lat_slice,
        chunks=chunks)["10u"]["u10"].chunk({"lat":-1,"lon":-1})
era5_vas = load_model_data.load_era5_variable(
        ["10v"],
        t1,
        t2,
        lon_slice,
        lat_slice,
        chunks=chunks)["10v"]["v10"].chunk({"lat":-1,"lon":-1})
era5_2d = load_model_data.load_era5_variable(
        ["2d"],
        t1,
        t2,
        lon_slice,
        lat_slice,
        chunks=chunks)["2d"]["d2m"]
era5_ps = load_model_data.load_era5_variable(
        ["sp"],
        t1,
        t2,
        lon_slice,
        lat_slice,
        chunks=chunks)["sp"]["sp"]
era5_huss = mpcalc.specific_humidity_from_dewpoint(era5_ps,era5_2d)  

In [ ]:
angle_ds = load_model_data.get_coastline_angle_kernel(
    lsm,
    compute=False,
    lat_slice=lat_slice,
    lon_slice=lon_slice,
    path_to_load="/g/data/gb02/ab4502/coastline_data/era5_global_angles.nc")


In [ ]:
q=era5_huss
import dask.array as da
u=era5_uas
v=era5_vas

#Rechunk data in one lat and lon dim
q = q.chunk({"lat":-1,"lon":-1})
u = u.chunk({"lat":-1,"lon":-1})
v = v.chunk({"lat":-1,"lon":-1})

#Convert specific humidity to g/kg
q = q*1000

#Calculate grid spacing in km using metpy, in x and y
x, y = np.meshgrid(q.lon,q.lat)
dx, dy = mpcalc.lat_lon_grid_deltas(x,y)

#Convert the x and y grid spacing arrays into xarray datasets. Need to interpolate to match the original grid
dx = xr.DataArray(np.array(dx),dims=["lat","lon"],coords={"lat":q.lat.values, "lon":q.lon.values[0:-1]}).\
        interp({"lon":q.lon,"lat":q.lat},method="linear",kwargs={"fill_value":"extrapolate"}).\
        chunk({"lat":q.chunksizes["lat"][0], "lon":q.chunksizes["lon"][0]})
dy = xr.DataArray(np.array(dy),dims=["lat","lon"],coords={"lat":q.lat.values[0:-1], "lon":q.lon.values}).\
        interp({"lon":q.lon,"lat":q.lat},method="linear",kwargs={"fill_value":"extrapolate"}).\
        chunk({"lat":q.chunksizes["lat"][0], "lon":q.chunksizes["lon"][0]})

#Calculate horizontal moisture gradient
ddy_q = (xr.DataArray(da.gradient(q,axis=q.get_axis_num("lat")), dims=q.dims, coords=q.coords) / dy)
ddx_q = (xr.DataArray(da.gradient(q,axis=q.get_axis_num("lon")), dims=q.dims, coords=q.coords) / dx)
mag_dq = np.sqrt( ddy_q**2 + ddx_q**2)

ddy_u = (xr.DataArray(da.gradient(u,axis=q.get_axis_num("lat")), dims=q.dims, coords=q.coords) / dy)
ddx_u = (xr.DataArray(da.gradient(u,axis=q.get_axis_num("lon")), dims=q.dims, coords=q.coords) / dx)
ddy_v = (xr.DataArray(da.gradient(v,axis=q.get_axis_num("lat")), dims=q.dims, coords=q.coords) / dy)
ddx_v = (xr.DataArray(da.gradient(v,axis=q.get_axis_num("lon")), dims=q.dims, coords=q.coords) / dx)
div = ddx_u + ddy_v
strch_def = ddx_u - ddy_v
shear_def = ddx_v + ddy_u
tot_def = np.sqrt(strch_def**2 + shear_def**2)

#Calculate the angle between axis of dilitation and isentropes
psi = 0.5 * np.arctan2(shear_def, strch_def)
beta = np.arcsin((-ddx_q * np.cos(psi) - ddy_q * np.sin(psi)) / mag_dq)

F = 0.5 * mag_dq * (tot_def * np.cos(2 * beta) - div) * 1.08e9

In [ ]:
F.persist()